In [26]:
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
import os

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,  # Можно временно изменить на DEBUG для отладки
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("update_status_p.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)


def fetch_data():
    """
    Извлекает данные из таблиц OlapCube_VNV и Vygruzka для расчета Status_P.
    """
    try:
        # Извлечение данных из OlapCube_VNV (учитываем первую дату)
        query_olap = f"""
        SELECT 
            serialno, location, ac_typ, ll, oh, BR, daily_flight_hours, MIN(Dates) AS first_date
        FROM {database_name}.OlapCube_VNV
        GROUP BY serialno, location, ac_typ, ll, oh, BR, daily_flight_hours
        """
        olap_data = client.execute(query_olap)

        # Извлечение данных из Vygruzka
        query_vygruzka = f"""
        SELECT 
            serialno, sne, ppr, condition 
        FROM {database_name}.Vygruzka
        """
        vygruzka_data = client.execute(query_vygruzka)

        # Преобразование Vygruzka в словарь для быстрого поиска по serialno
        vygruzka_dict = {row[0]: {'sne': row[1], 'ppr': row[2], 'condition': row[3]} for row in vygruzka_data}

        logger.info(f"Извлечено {len(olap_data)} записей из OlapCube_VNV и {len(vygruzka_data)} записей из Vygruzka.")
        return olap_data, vygruzka_dict
    except Exception as e:
        logger.error(f"Ошибка при извлечении данных: {e}", exc_info=True)
        return None, None


def calculate_status_p(olap_data, vygruzka_dict):
    """
    Рассчитывает Status_P для каждой записи на основе условий.
    """
    status_updates = []
    unresolved = []

    for row in olap_data:
        serialno, location, ac_typ, ll, oh, BR, daily_flight_hours, first_date = row

        # Пропускаем записи с serialno, начинающимся на "S"
        if serialno.startswith("S"):
            logger.warning(f"Пропускаем запись с serialno: {serialno}, начинается на 'S'.")
            continue

        vygruzka_entry = vygruzka_dict.get(serialno, None)

        if not vygruzka_entry:
            logger.warning(f"Не найдены данные Vygruzka для serialno: {serialno}")
            unresolved.append(serialno)
            continue

        sne = vygruzka_entry['sne']
        ppr = vygruzka_entry['ppr']
        condition = vygruzka_entry['condition']

        # Проверяем значения sne и ppr на наличие None
        if sne is None or ppr is None or ll is None or oh is None or BR is None:
            logger.warning(f"Пропускаем запись с serialno: {serialno} из-за отсутствия данных.")
            unresolved.append(serialno)
            continue

        # Приведение типов и обработка значений
        try:
            sne = float(sne)
            ppr = float(ppr)
            ll = float(ll)
            oh = float(oh)
            BR = float(BR)
            daily_flight_hours = float(daily_flight_hours)
        except ValueError as ve:
            logger.warning(f"Некорректные числовые значения для serialno: {serialno}. Ошибка: {ve}")
            unresolved.append(serialno)
            continue

        # Применение условий в правильном порядке
        if location and location.startswith("RA-") and ac_typ in ("Ми-8Т", "Ми-17"):
            # Эксплуатация
            if (sne < (ll - daily_flight_hours)) and (ppr < (oh - daily_flight_hours)) and (condition == "ИСПРАВНЫЙ"):
                status = "Эксплуатация"
            else:
                unresolved.append(serialno)
                continue
        elif (sne < (ll - daily_flight_hours)) and (ppr < (oh - daily_flight_hours)) and (condition == "ИСПРАВНЫЙ"):
            # Исправен
            status = "Исправен"
        elif (sne < BR) and (condition == "НЕИСПРАВНЫЙ"):
            # Ремонт
            status = "Ремонт"
        elif (sne >= BR) and (condition == "НЕИСПРАВНЫЙ"):
            # Хранение
            status = "Хранение"
        else:
            unresolved.append(serialno)
            continue

        status_updates.append((status, serialno, first_date))
        logger.debug(f"serialno: {serialno}, calculated Status_P: {status}, first_date: {first_date}")

    if unresolved:
        logger.error(f"Не удалось определить Status_P для следующих serialno: {unresolved}")

    return status_updates


def update_status_p(status_updates):
    """
    Обновляет поле Status_P в таблице OlapCube_VNV.
    """
    try:
        for status, serialno, first_date in status_updates:
            update_query = f"""
            ALTER TABLE {database_name}.OlapCube_VNV 
            UPDATE Status_P = %(status)s 
            WHERE serialno = %(serialno)s AND Dates = %(first_date)s
            """
            params = {
                'status': status,
                'serialno': serialno,
                'first_date': first_date
            }
            try:
                client.execute(update_query, params)
            except Exception as e_inner:
                logger.error(f"Ошибка при обновлении Status_P для serialno: {serialno}, Dates: {first_date}. Ошибка: {e_inner}", exc_info=True)
                continue  # Продолжаем обработку остальных записей
        logger.info(f"Обновлено {len(status_updates)} записей в OlapCube_VNV.")
    except Exception as e:
        logger.error(f"Ошибка при обновлении Status_P: {e}", exc_info=True)


def main():
    # Шаг 1: Извлечение данных
    olap_data, vygruzka_dict = fetch_data()
    if not olap_data or not vygruzka_dict:
        logger.error("Не удалось извлечь данные. Завершаем выполнение.")
        return

    # Шаг 2: Расчет Status_P
    status_updates = calculate_status_p(olap_data, vygruzka_dict)

    # Шаг 3: Обновление поля Status_P
    update_status_p(status_updates)


if __name__ == "__main__":
    main()


2024-12-02 09:31:25,738 - INFO - Извлечено 420 записей из OlapCube_VNV и 185 записей из Vygruzka.
2024-12-02 09:31:25,739 - WARNING - Пропускаем запись с serialno: S122, начинается на 'S'.
2024-12-02 09:31:25,742 - WARNING - Пропускаем запись с serialno: S29, начинается на 'S'.
2024-12-02 09:31:25,743 - WARNING - Пропускаем запись с serialno: S24, начинается на 'S'.
2024-12-02 09:31:25,744 - WARNING - Пропускаем запись с serialno: S92, начинается на 'S'.
2024-12-02 09:31:25,745 - WARNING - Пропускаем запись с serialno: S67, начинается на 'S'.
2024-12-02 09:31:25,746 - WARNING - Пропускаем запись с serialno: S180, начинается на 'S'.
2024-12-02 09:31:25,747 - WARNING - Пропускаем запись с serialno: S150, начинается на 'S'.
2024-12-02 09:31:25,749 - WARNING - Пропускаем запись с serialno: S221, начинается на 'S'.
2024-12-02 09:31:25,750 - WARNING - Пропускаем запись с serialno: S40, начинается на 'S'.
2024-12-02 09:31:25,751 - WARNING - Пропускаем запись с serialno: S70, начинается на 'S'